In [12]:
import glob
import json
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt

In [13]:
thresholds = [0.2, 0.5, 1, 2, 5]
window_sizes = [4, 7, 14, 21]

### continuous metrics

In [14]:
cli = np.load("/p/project/deepacf/deeprain/wittenbrink/py/sample_climatology.npy")

def leps(y1,y2,climato):
    if len(y1.shape)==0:
        return abs(np.sum(y1>climato) - np.sum(y2>climato))/len(climato)
    else:
        return np.mean([leps(y1[i],y2[i],climato) for i in range(len(y1))])

In [15]:
from sklearn.metrics import mean_absolute_error

### categorical metrics

In [16]:
def booleanize(array, threshold):
    booleanized = array >= threshold
    return booleanized.astype(int) 

In [17]:
def contingencytable(veri,perc,threshh):
    hits=np.sum((veri>=threshh)*(perc>=threshh))
    falsealarms=np.sum((veri<threshh)*(perc>=threshh))
    misses=np.sum((veri>=threshh)*(perc<threshh))
    correctnegatives=np.sum((veri<threshh)*(perc<threshh))
    return hits, falsealarms, misses, correctnegatives

def freqbias(hits,falsealarms, misses, correctnegatives):
    return (hits+falsealarms)/(hits+misses)

def ets(hits,falsealarms, misses, correctnegatives):
    hitsrandom=(hits+misses)*(hits+falsealarms)/np.sum((hits,falsealarms, misses, correctnegatives))
    return (hits-hitsrandom)/(hits+misses+falsealarms-hitsrandom)

def lor(hits,falsealarms, misses, correctnegatives):
    return np.log((hits*correctnegatives)/(misses*falsealarms))

def pod(hits,falsealarms, misses, correctnegatives):
    return hits/(hits+misses)

def far(hits,falsealarms, misses, correctnegatives):
    return falsealarms/(falsealarms+hits)

def csi(hits,falsealarms, misses, correctnegatives):
    return hits/(hits+falsealarms+misses)

## Evaluating predictions

### cosmo

In [ ]:
cosmo = np.load('/p/scratch/deepacf/deeprain/rojascampos1/data/radar_enhancement/lres/01_tst_c.npy')
print('cosmo.shape =', cosmo.shape)

tst_y_lres = np.load('/p/scratch/deepacf/deeprain/rojascampos1/data/radar_enhancement/lres/01_tst_y.npy')
print('tst_y_lres.shape =', tst_y_lres.shape)

results_cosmo = {}

results_cosmo['mae'] = mean_absolute_error(tst_y_lres.flatten(), cosmo.flatten())
results_cosmo['leps'] = leps(tst_y_lres.flatten(), cosmo.flatten(), cli)

results_cosmo['freqbias'] = {}
results_cosmo['ets'] = {}
results_cosmo['lor'] = {}
results_cosmo['pod'] = {}
results_cosmo['far'] = {}
results_cosmo['csi'] = {}
results_cosmo['fss'] = {}

for t in thresholds:

    hits, falsealarms, misses, correctnegatives = contingencytable(tst_y_lres, cosmo, t)
    results_cosmo['freqbias'][str(t)] = freqbias(hits, falsealarms, misses, correctnegatives)
    results_cosmo['ets'][str(t)] = ets(hits, falsealarms, misses, correctnegatives)
    results_cosmo['lor'][str(t)] = lor(hits, falsealarms, misses, correctnegatives)
    results_cosmo['pod'][str(t)] = pod(hits, falsealarms, misses, correctnegatives)
    results_cosmo['far'][str(t)] = far(hits, falsealarms, misses, correctnegatives)
    results_cosmo['csi'][str(t)] = csi(hits, falsealarms, misses, correctnegatives)
    results_cosmo['fss'][str(t)] = {}

    for w in window_sizes:
        results_cosmo['fss'][str(t)][str(w)] = fss(tst_y_lres, cosmo, t, w, False)

with open('results_cosmo.json', 'w') as fp:
    json.dump(results_cosmo, fp)

### Baseline, UNet, Deconv1L, Deconv3L, CGans

In [19]:
def calcsadscores(obs,forc):
    verif = sad( 144, 4, True, threshold )
    if len(obs)==len(forc):
        sads=[verif.verify(obs[i],forc[i]) for i in range(len(obs))]
        return np.array(sads)[:,0,:]
    else:
        
        sads=[verif.verify(obs[i],*[forc[j][i] for j in range(len(forc))]) for i in range(len(obs))]
        return np.array(sads)

In [20]:
def evaluate_models(path):
    
    tst_x = np.load('/p/scratch/deepacf/deeprain/rojascampos1/data/radar_enhancement/hres/01_tst_x.npy')
    tst_y = np.load('/p/scratch/deepacf/deeprain/rojascampos1/data/radar_enhancement/hres/01_tst_y.npy')
    
    models = sorted(glob.glob(path + 'models/*.h5'))
    
    all_preds = []
    results = {}
    
    for idx, m in enumerate(models):
        
        print(m)
        mo = tf.keras.models.load_model(m)
        preds = mo(tst_x)
        preds = np.squeeze(preds.numpy())
        all_preds.append(preds)
        
        results_m = {}
        
        results_m['mae'] = mean_absolute_error(tst_y.flatten(), preds.flatten())
        results_m['leps'] = leps(tst_y.flatten(), preds.flatten(), cli)
        
        results_m['freqbias'] = {}
        results_m['ets'] = {}
        results_m['lor'] = {}
        results_m['pod'] = {}
        results_m['far'] = {}
        results_m['csi'] = {}
        results_m['fss'] = {}
        
        for t in thresholds:
            
            hits, falsealarms, misses, correctnegatives = contingencytable(tst_y.flatten(), preds.flatten(), t)
            results_m['freqbias'][str(t)] = freqbias(hits, falsealarms, misses, correctnegatives)
            results_m['ets'][str(t)] = ets(hits, falsealarms, misses, correctnegatives)
            results_m['lor'][str(t)] = lor(hits, falsealarms, misses, correctnegatives)
            results_m['pod'][str(t)] = pod(hits, falsealarms, misses, correctnegatives)
            results_m['far'][str(t)] = far(hits, falsealarms, misses, correctnegatives)
            results_m['csi'][str(t)] = csi(hits, falsealarms, misses, correctnegatives)
            results_m['fss'][str(t)] = {}
            
            for w in window_sizes:
                results_m['fss'][str(t)][str(w)] = fss(tst_y, preds, t, w, False)
        
        results[str(idx)] = results_m

    ## Save results
    np.save(path + '/predictions.npy', np.array(all_preds))
    with open(path + '/results.json', 'w') as fp:
        json.dump(results, fp)

In [ ]:
evaluate_models('/p/home/jusers/rojascampos1/juwels/MyProjects/PROJECT_deepacf/deeprain/rojascampos1/radar_enhancement/scripts/baseline/')

In [ ]:
evaluate_models('/p/home/jusers/rojascampos1/juwels/MyProjects/PROJECT_deepacf/deeprain/rojascampos1/radar_enhancement/scripts/unet/')

In [ ]:
evaluate_models('/p/home/jusers/rojascampos1/juwels/MyProjects/PROJECT_deepacf/deeprain/rojascampos1/radar_enhancement/scripts/deconv1l/')

In [ ]:
evaluate_models('/p/home/jusers/rojascampos1/juwels/MyProjects/PROJECT_deepacf/deeprain/rojascampos1/radar_enhancement/scripts/deconv3l/')

In [ ]:
evaluate_models('/p/home/jusers/rojascampos1/juwels/MyProjects/PROJECT_deepacf/deeprain/rojascampos1/radar_enhancement/scripts/cgan/')